### Preprocessing the Yelp Comments on Berlin Restaurants

#### Get required packages

In [1]:
import pandas as pd
import re
import nltk
import string
import nltk.tokenize
#Run in Terminal
#pip install spacy
#python -m spacy download de_core_news_md
import spacy as spacy
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('stopwords')
stop_words = nltk.corpus.stopwords.words('german')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ana\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ana\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


#### Get dataset

In [2]:
yelp = pd.read_csv('../1_scraping/intermediary_outputs/german_merged.csv')
yelp.columns.values[0]="ID"
yelp = yelp.rename(columns={'Overall Rating':'Overall_Rating',
                                      "Total Reviews":'Total_Reviews',
                                      "Restaurant Name":"Restaurant_Name",
                                     "Price Range":"Price_Range"})
yelp.columns
yelp['ID'] = yelp['ID'].astype(int)
yelp['Comment'] = yelp['Comment'].astype(str)
yelp['Overall_Rating'] = yelp['Overall_Rating'].astype(float)
yelp['Date'] = yelp['Date'].astype('datetime64[ns]',"dd-MM-yyyy")
yelp.dtypes
print(yelp.info())



C:\Users\ana\AppData\Local\Temp\ipykernel_11344\236994953.py:11: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  yelp['Date'] = yelp['Date'].astype('datetime64[ns]',"dd-MM-yyyy")


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9721 entries, 0 to 9720
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   ID               9721 non-null   int32         
 1   Restaurant_Name  9721 non-null   object        
 2   Overall_Rating   9721 non-null   float64       
 3   Total_Reviews    9721 non-null   int64         
 4   Specialty        9721 non-null   object        
 5   Region           9721 non-null   object        
 6   Price_Range      9721 non-null   object        
 7   Author           9721 non-null   object        
 8   Comment          9721 non-null   object        
 9   Rating           9721 non-null   int64         
 10  Date             9721 non-null   datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int32(1), int64(2), object(6)
memory usage: 797.6+ KB
None


#### Remove Duplicates

In [3]:
print(len(yelp))
# Remove duplicates
yelp = yelp.drop_duplicates(subset=['Comment'], inplace=False)
print(len(yelp)) #15 duplicates found

9721
9706


#### Basic Cleaning

In [4]:
#remove pattern "Unknown" / "x Fotos" in Author - replace with None
yelp['Author'] = yelp.Author.where((yelp.Author == 'Unknown') | ('Foto' in yelp.Author), None)
yelp['Price_Range'] = yelp.Price_Range.where(yelp.Price_Range == 'Unknown', None)
#print(yelp[yelp.ID==1114])

def clean_text(text):
    #fix enter instead of punctuation or space but keep contiguous capitalization
    text = re.sub('(?<=.)(?=[A-Z][a-z])', r" ", text)
    #remove random urls w/o http
    text = re.sub('[\w]+\.[\w]+\/+[\w]+','',text)
    #remove URL with http
    text = re.sub(r'http\S+', '', text)
    #remove URL with www
    text = re.sub(r'www\S+', '', text)
    # add space after . to avoid word concatenation when user left no space after .
    text = re.sub(r'(?<=[.,)!])(?=[^\s])', r' ', text)
    # Remove Emoji chars
    emoticons = r'[\W]+(?::|;|=)(?:-)?(?:\)+|\(|D|P)'
    text = re.sub(emoticons, '' , text)
    #remove hashtags but leave the content of a hashtag in text
    text = re.sub(r"([#]+)", "", text)
    #remove @name shoutouts +  weird shoutouts with space between @ and name
    pattern_shoutout_one = r"((\w+|[^a-z])[@](\s+\w+|\w+|.*))"
    text = re.sub(pattern_shoutout_one, "",text)
    #Remove weird unicode characters such as U+2026
    text = re.sub(r'[^\x00-\x7FäöüÄÖÜß]+', '', text)
    # # remove hashtags and normal shoutouts with @
    # pattern_hashtags_shoutouts = r"([@#]\w+)"
    # text = re.sub(pattern_hashtags_shoutouts,"" ,text)

    #remove digits
    text = re.sub(r'[0-9]+', '', text)
    #remove 1 char words
    text = re.sub(r'\b\w{1}\b', ' ', text)
    #Remove punctuations etc.
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    #Convert to lowercase to maintain consistency
    text = text.lower()
    return text

df = yelp[['ID', 'Comment']]
pd.set_option('max_colwidth', 800)


yelp['Comment'] = yelp.Comment.apply(lambda x: clean_text(x))
print("URL case")
print(yelp.Comment[0])
print(yelp.Comment[3562])
print("Emoji case")
print(yelp.Comment[28])
print(yelp.Comment[6])
print(yelp.Comment[226])
print(yelp.Comment[164])
print("Shoutouts")
print("1")
print(yelp.Comment[278])#remove comments in english
print("2")
print(yelp.Comment[2672])
print("3")
print(yelp.Comment[6434])
print("4")
print(yelp.Comment[2690])
print("Hashtags")
print(yelp.Comment[1134])#remove comments in english
print(yelp.Comment[4605])
print("Commentar 112 gelandetTolles")
print(yelp.Comment[112])
print(yelp.Comment[2680])

URL case
ich habe mir  bewertungen zu  restaurants auf  menulist angesehen und dieses  restaurant hatte gute  kritiken also habe ich es ausprobiert und es nicht bereut 
neue  homepage
Emoji case
der wohl verrückteste  kellner den ich je hatte aber eine einzigartige  kneipe die sich  hinter der  stadtklause versteckt  im  kellergewölbe kann man auch noch sitzen günstiges deutsches  essen und  bier in einem eigentlich so touristisch überfluteten  potsdamer  platz  ecke gerade bei den sonst so hohen  bierpreisen in der  umgebung des  sony  center liefert die  stadtklause eine  entspannung für den  geldbeutel  preis leistung stimmt
super leckeres  essen cooles  atmosphäre  gutes  bier leider keine ec  karten oder  kreditkarten aktzeptiert das ist ein bissle schade
mit  abstand das beste  schnitzel weit und breit  super nettes  ambiente und tolles  personal beim nächsten berlin besuch wieder  daumen hoch
beste  ramen die ich jemals gegessen habe ja es ist voll aber man wartet mi ich lange a

In [5]:
yelp.head()

,ID,Restaurant_Name,Overall_Rating,Total_Reviews,Specialty,Region,Price_Range,Author,Comment,Rating,Date
0,0,JOSEPH,4.5,6,Israelisch /,Mitte,Unknown,None,ich habe mir bewertungen zu restaurants auf menulist angesehen und dieses restaurant hatte gute kritiken also habe ich es ausprobiert und es nicht bereut,5,2022-10-26
1,1,JOSEPH,4.5,6,Israelisch /,Mitte,Unknown,None,das joseph ist ein ziemlich neues israelisches restaurant im modernen design es ist zwar nicht besonders kuschelig aber angenehm hell und hübsch eingerichtet mit schönen lampen der service ist ausgesprochen freundlich unkompliziert und aufmerksam besonders celina das essen kam schnell sah sehr appetitlich aus und so schmeckte es auch die israelischen starter kleinigkeiten und hauptspeisen haben uns sehr gut geschmeckt die preise sind angemessen und für berlin mitte nicht zu teuer wir waren nur etwas überrascht dass schon um sehr laute musik aufgelegt wurde wir konnten uns fast nicht mehr unterhalten am er tisch gegenüber auf anfrage wurde das etwas gedrosselt fazit wer israelische küche mag gutes preis leistungs verhältnis möchte mit blick auf die friedri...,4,2019-07-11
2,2,JOSEPH,4.5,6,Israelisch /,Mitte,Unknown,None,schalömchen scheiße haben wir uns heute koscher weggeknallt maria und joseph werden zum vorbild kellerinnen geben alles und saufen mehr als der gast als ich rein kam schlägt es wellen aber ich bin diesen sommer einfach lit as fuck und alle saufen mit mir ne geile crew ist die halbe miete hätte gerne mehr geferkelt aber war zu dicht,5,2019-10-18
3,3,JOSEPH,4.5,6,Israelisch /,Mitte,Unknown,None,essen war allenfalls ok muss ich nicht nochmal haben insgesamt war mein erlebnis im joseph sehr langweilig nachdem ich allein ins restaurant kam dauerte es erstmal bis ich begrüßt und platziert wirde das lokal war nicht voll die speisekarte auf englisch was mich in berlin mitte nicht wirklich verwundert ich bestellte einen grüne bohnen salat als vorspeise und danach einen grillspieß mit gemüse in der offenen küche zubereitet kam es nach wenigen minuten am tisch an sellerie und walnüsse die lt karte unter anderem in meinem wohl klingenden salat sein sollten fehlten der selbstgemachte bagel als beilage zum salat hingegen hat mir gut geschmeckt der grillspieß bestand aus kartoffel zwiebel aubergine paprika und zucchini gebettet auf stellenweise schwarz ...,2,2020-08-01
4,4,JOSEPH,4.5,6,Israelisch /,Mitte,Unknown,None,waren auf empfehlung dort und wurden nicht enttäuscht hammer ambiente hammer essen mega personal wir waren zu da und haben uns was zusammen stellen lassen einmal alles ich als vegetarier bin auch voll auf meine kosten gekommen es macht wirklich sinn und so ist auch das konzept das man mehrere gerichte bestellt und alles untereinander teilt,5,2019-03-09


#### Tokenization & Stopword Removal

In [6]:

def tokenize_and_remove_stopword(text):
    tokens = word_tokenize(text,language='german')
    text = " ".join([word for word in tokens if word not in stop_words])
    return text

yelp['Comment'] = yelp['Comment'].apply(lambda x: tokenize_and_remove_stopword(x))
print(yelp.Comment[7890])
print(yelp.Comment[112])

monate schnitzelei großen schnitzel lecker karte kreativ zusammengestellt insbesondere kostenlose begrüßungsbier deutschen tapas heben schnitzelei nochmal lokalen ab zudem bedienungen freundlich ambiente angenehm
zufall marral gelandet tolles essen toller service guten preis kommen defintiv


#### Lemmatisierung mit SpaCy

In [7]:
# https://blog.codecentric.de/natural-language-processing-basics
# https://textmining.wp.hs-hannover.de/Preprocessing.html
# https://nickyreinert.de/blog/2020/12/09/einfuehrung-in-stemming-und-lemmatisierung-deutscher-texte-mit-python/
# https://de.steadforce.com/blog/natural-language-processing-tools
# https://spacy.io/usage/linguistic-features#lemmatization

In [8]:
#pip install spacy
#python -m spacy download de_core_news_md

In [9]:
spc =  spacy.load(r'de_core_news_md')

def lemmatize_spc(tokenized_comment):

    tok_cmt_as_spacy_object = spc(tokenized_comment)
    text = " ".join([token.lemma_ for token in tok_cmt_as_spacy_object])
    return text.lower()
yelp['Comment'] = yelp['Comment'].apply(lambda x: lemmatize_spc(x))





#### Drop Columns and Save to CSV

In [10]:
yelp.drop(["ID", "Restaurant_Name", "Overall_Rating", "Total_Reviews", "Specialty", "Region", "Author", "Date", "Price_Range"],inplace = True,axis = 1)
yelp.to_csv("cleaned_data_yelp.csv", index=False)